# Start of DS5559 Final Project

Team Left Twix Members

* Alice Wright - aew7j
* Edward Thompson - ejt8b
* Michael Davies -  mld9s
* Sam Parsons - sp8hp

In STAT 6021 members of our cohort looked at Transportation Network Company data sets to see if there was a potential relationship between tipping and other indicators, specifically with “transportation network providers” i.e. rideshares such as Uber, Lyft, etc.  At that point in our Data Science journey we did not have the skills or equipment to investigate this question in depth.  

Utilizing machine learning skills from SYS 6018 and applying Spark to this dataset we hope to come up with a more robust set of answers and potentially a better predictor of tipping. With other classification algorithms such as random forest and the heavy-weight data processing of Spark, will we be able to create a more robust predictive model?


Potential Questions from the TNC Data:

* Can it be predicted what fares are most likely to tip the driver?
* Is there a relationship between time of the fare and tipping? (workday stat, bar close, weekday, weekend, etc)
* Is there a relationship between start or end location of the ride and tipping? (downtown pickup, north shore, airport, etc)
* Is there a relationship between length or cost of ride and tipping? (do longer rides result in tips)
* Using this data would we be able to make recommendations to drivers to maximize likelihood of receiving a tip?
* Is the likelihood of tipping changing over time?  Are more rides being tipped?
* Are there re-identification abilities in this dataset? For instance, can we find records for a person who reliably takes a rideshare to/from work every day thereby linking a home address to a work address?




Additionally, joining in additional datasets may yield answers to questions about external factors such as:
* How did news reporting/social media on rideshare companies correlate with tipping?
* What relationship(s) does trip demand have with the stocks of these companies?

Data Source:
The best data source for this appears to be from the City of Chicago, as it is large (169M records and 21 columns), relatively clean, anonymized, and accessible via API.

City of Chicago:
https://data.cityofchicago.org/Transportation/Transportation-Network-Providers-Trips/m6dm-c72p

So far we have only pulled the data down via a CSV.

Code Rubric

* Data Import and PreProcessing | 2 pts

* Data splitting/sampling | 1 pt

* EDA (min two graphs) | 2 pts

* Model construction (min 3 models) | 3 pts

* Model evaluation | 2 pts

In [1]:
# import context manager: SparkSession
from pyspark.sql import SparkSession

# import data types
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType, BooleanType
import pyspark.sql.types as typ
import pyspark.sql.functions as F
import os



from pyspark.sql.types import *

spark = SparkSession.builder \
        .master("local[*]") \
        .appName("mllib_classifier") \
        .config("spark.executor.memory", '21g') \
        .config('spark.executor.cores', '2') \
        .config('spark.executor.instances', '3') \
        .config("spark.driver.memory",'1g') \
        .getOrCreate()
sc = spark.sparkContext

# import data manipulation methods
from pyspark.ml.feature import Binarizer
from pyspark.ml import Pipeline  
from pyspark.ml.feature import *  
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.classification import GBTClassifier

from pyspark.ml.linalg import DenseVector
from pyspark.ml.feature import VectorAssembler 
from pyspark.mllib.linalg import Vectors

from pyspark.ml.feature import StandardScaler
from pyspark.ml.feature import OneHotEncoder
#from pyspark.ml.feature import IndexToString, StringIndexer, VectorIndexer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.evaluation import BinaryClassificationEvaluator

from pyspark.mllib.evaluation import MulticlassMetrics
from pyspark.mllib.evaluation import BinaryClassificationMetrics

from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

import time

import numpy as np

In [2]:
%whos

Variable                            Type            Data/Info
-------------------------------------------------------------
ArrayType                           type            <class 'pyspark.sql.types.ArrayType'>
Binarizer                           type            <class 'pyspark.ml.feature.Binarizer'>
BinaryClassificationEvaluator       type            <class 'pyspark.ml.evalua<...>ClassificationEvaluator'>
BinaryClassificationMetrics         type            <class 'pyspark.mllib.eva<...>ryClassificationMetrics'>
BinaryType                          type            <class 'pyspark.sql.types.BinaryType'>
BooleanType                         type            <class 'pyspark.sql.types.BooleanType'>
BucketedRandomProjectionLSH         type            <class 'pyspark.ml.featur<...>etedRandomProjectionLSH'>
BucketedRandomProjectionLSHModel    type            <class 'pyspark.ml.featur<...>andomProjectionLSHModel'>
Bucketizer                          type            <class 'pyspark.ml.feature.B

In [3]:
#clear old df
#del (df)

# Read in our Dataset

## Create a Custom Schema.  
This schema was been primarly determined by using a much smaller dataset and letting spark infer the schema.  We encountered an issue with spark reading in the ENTIRE dataset as NULL when there was a type mismatch.  Only the data we are likely to use later has been assigned to a specific type, otherwise it is left as a string type.

In [4]:
# create a custom schema.  

customSchema = StructType([
    StructField('Trip_ID', StringType(), True),        
    StructField('Trip_Start_Timestamp', StringType(), True),
    StructField('Trip_End_Timestamp', StringType(), True),
    StructField('Trip_Seconds', DoubleType(), True),
    StructField('Trip_Miles', DoubleType(), True),
    StructField('Pickup_Census_Tract', StringType(), True),
    StructField('Dropoff_Census_Tract', StringType(), True),
    StructField('Pickup_Community_Area', DoubleType(), True),
    StructField('Dropoff_Community_Area', DoubleType(), True),
    StructField("Fare", DoubleType(), True),
    StructField("Tip", DoubleType(), True),
    StructField("Additional_Charges", DoubleType(), True),
    StructField("Trip_Total", StringType(), True),
    StructField("Shared_Trip_Authorized", BooleanType(), True),
    StructField("Trips_Pooled", DoubleType(), True),
    StructField('Pickup_Centroid_Latitude', StringType(), True),
    StructField('Pickup_Centroid_Longitude', StringType(), True),
    StructField('Pickup_Centroid_Location', StringType(), True),
    StructField('Dropoff_Centroid_Latitude', StringType(), True),
    StructField('Dropoff_Centroid_Longitude', StringType(), True),
    StructField('Dropoff_Centroid_Location', StringType(), True)])

#old readin.  Infer is slow for large dataset
#df = spark.read.csv('/../../project/ds5559/Alice_Ed_Michael_Sam_project/BigTrips.csv', header = True, inferSchema=True)

#read in the data to a dataframe
df = spark.read.csv('/../../project/ds5559/Alice_Ed_Michael_Sam_project/BigTrips.csv', header = True, schema=customSchema)
df.show(5)

+--------------------+--------------------+--------------------+------------+----------+-------------------+--------------------+---------------------+----------------------+----+---+------------------+----------+----------------------+------------+------------------------+-------------------------+------------------------+-------------------------+--------------------------+-------------------------+
|             Trip_ID|Trip_Start_Timestamp|  Trip_End_Timestamp|Trip_Seconds|Trip_Miles|Pickup_Census_Tract|Dropoff_Census_Tract|Pickup_Community_Area|Dropoff_Community_Area|Fare|Tip|Additional_Charges|Trip_Total|Shared_Trip_Authorized|Trips_Pooled|Pickup_Centroid_Latitude|Pickup_Centroid_Longitude|Pickup_Centroid_Location|Dropoff_Centroid_Latitude|Dropoff_Centroid_Longitude|Dropoff_Centroid_Location|
+--------------------+--------------------+--------------------+------------+----------+-------------------+--------------------+---------------------+----------------------+----+---+-------

In [5]:
#Doesn't update if you don't resave the variable

df = df.drop('Pickup_Census_Tract',
             'Dropoff_Census_Tract',
             'Pickup_Centroid_Latitude',
             'Pickup_Centroid_Longitude', 
             'Pickup_Centroid_Location', 
             'Dropoff_Centroid_Latitude', 
             'Dropoff_Centroid_Longitude', 
             'Dropoff_Centroid_Location')

#'Trip_End_Timestamp' keep for now

In [6]:
df.printSchema()

root
 |-- Trip_ID: string (nullable = true)
 |-- Trip_Start_Timestamp: string (nullable = true)
 |-- Trip_End_Timestamp: string (nullable = true)
 |-- Trip_Seconds: double (nullable = true)
 |-- Trip_Miles: double (nullable = true)
 |-- Pickup_Community_Area: double (nullable = true)
 |-- Dropoff_Community_Area: double (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Tip: double (nullable = true)
 |-- Additional_Charges: double (nullable = true)
 |-- Trip_Total: string (nullable = true)
 |-- Shared_Trip_Authorized: boolean (nullable = true)
 |-- Trips_Pooled: double (nullable = true)



In [7]:
df2 = df.sample(False, .0005, seed = 2021) #decreased our sample size

In [8]:
df2.count()

24478

In [9]:
#delete the big df for now
del (df)

#hopefully that will make things faster 

In [10]:
#fill our NA community areas

df2 = df2.na.fill(value=78,subset=['Pickup_Community_Area', 'Dropoff_Community_Area'])

In [11]:
# make a binary tip/no tip indicator
# https://spark.apache.org/docs/2.2.0/ml-features.html#binarizer

#binarized tip seems to be causing problems.  Change its name to label as that is that the packages are expecting

binarizer = Binarizer(threshold=0, inputCol="Tip", outputCol="label")
df2 = binarizer.transform(df2)

In [12]:
df2.printSchema()

root
 |-- Trip_ID: string (nullable = true)
 |-- Trip_Start_Timestamp: string (nullable = true)
 |-- Trip_End_Timestamp: string (nullable = true)
 |-- Trip_Seconds: double (nullable = true)
 |-- Trip_Miles: double (nullable = true)
 |-- Pickup_Community_Area: double (nullable = false)
 |-- Dropoff_Community_Area: double (nullable = false)
 |-- Fare: double (nullable = true)
 |-- Tip: double (nullable = true)
 |-- Additional_Charges: double (nullable = true)
 |-- Trip_Total: string (nullable = true)
 |-- Shared_Trip_Authorized: boolean (nullable = true)
 |-- Trips_Pooled: double (nullable = true)
 |-- label: double (nullable = true)



In [13]:
df2 = df2.withColumn("Trip_Start_TS", F.to_timestamp(F.col("Trip_Start_Timestamp"), "MM/dd/yyyy hh:mm:ss a"))

In [14]:
df2.printSchema()

root
 |-- Trip_ID: string (nullable = true)
 |-- Trip_Start_Timestamp: string (nullable = true)
 |-- Trip_End_Timestamp: string (nullable = true)
 |-- Trip_Seconds: double (nullable = true)
 |-- Trip_Miles: double (nullable = true)
 |-- Pickup_Community_Area: double (nullable = false)
 |-- Dropoff_Community_Area: double (nullable = false)
 |-- Fare: double (nullable = true)
 |-- Tip: double (nullable = true)
 |-- Additional_Charges: double (nullable = true)
 |-- Trip_Total: string (nullable = true)
 |-- Shared_Trip_Authorized: boolean (nullable = true)
 |-- Trips_Pooled: double (nullable = true)
 |-- label: double (nullable = true)
 |-- Trip_Start_TS: timestamp (nullable = true)



In [15]:
df2 = df2.withColumn('Trip_Year',F.year(F.to_timestamp('Trip_Start_TS'))) \
         .withColumn('Trip_Month',F.month(F.to_timestamp('Trip_Start_TS'))) \
         .withColumn('Trip_WeekNumber',F.weekofyear(F.to_timestamp('Trip_Start_TS'))) \
         .withColumn('Trip_DayofWeek', F.dayofweek(F.col('Trip_Start_TS'))) \
         .withColumn('Trip_Start_Hour', F.hour(F.col('Trip_Start_TS'))) \
         .withColumn('Trip_Start_Minute', F.minute(F.col('Trip_Start_TS'))) \
         .withColumn('Date', F.to_date(F.col('Trip_Start_TS')))
         
df2.show(5, False)

+----------------------------------------+----------------------+----------------------+------------+----------+---------------------+----------------------+----+---+------------------+----------+----------------------+------------+-----+-------------------+---------+----------+---------------+--------------+---------------+-----------------+----------+
|Trip_ID                                 |Trip_Start_Timestamp  |Trip_End_Timestamp    |Trip_Seconds|Trip_Miles|Pickup_Community_Area|Dropoff_Community_Area|Fare|Tip|Additional_Charges|Trip_Total|Shared_Trip_Authorized|Trips_Pooled|label|Trip_Start_TS      |Trip_Year|Trip_Month|Trip_WeekNumber|Trip_DayofWeek|Trip_Start_Hour|Trip_Start_Minute|Date      |
+----------------------------------------+----------------------+----------------------+------------+----------+---------------------+----------------------+----+---+------------------+----------+----------------------+------------+-----+-------------------+---------+----------+---------

In [16]:
df2.printSchema()

root
 |-- Trip_ID: string (nullable = true)
 |-- Trip_Start_Timestamp: string (nullable = true)
 |-- Trip_End_Timestamp: string (nullable = true)
 |-- Trip_Seconds: double (nullable = true)
 |-- Trip_Miles: double (nullable = true)
 |-- Pickup_Community_Area: double (nullable = false)
 |-- Dropoff_Community_Area: double (nullable = false)
 |-- Fare: double (nullable = true)
 |-- Tip: double (nullable = true)
 |-- Additional_Charges: double (nullable = true)
 |-- Trip_Total: string (nullable = true)
 |-- Shared_Trip_Authorized: boolean (nullable = true)
 |-- Trips_Pooled: double (nullable = true)
 |-- label: double (nullable = true)
 |-- Trip_Start_TS: timestamp (nullable = true)
 |-- Trip_Year: integer (nullable = true)
 |-- Trip_Month: integer (nullable = true)
 |-- Trip_WeekNumber: integer (nullable = true)
 |-- Trip_DayofWeek: integer (nullable = true)
 |-- Trip_Start_Hour: integer (nullable = true)
 |-- Trip_Start_Minute: integer (nullable = true)
 |-- Date: date (nullable = true)


In [17]:
# split the data

# our model didn't work on the standard test train split.  Prof. Tashman recomended upscalling the help with the imbalanced dataset.
#from https://spark.apache.org/docs/2.1.0/ml-tuning.html#train-validation-split

train_inital, test = df2.randomSplit([0.8, 0.2], seed=2021)

# cahce our test values for later speed
test.cache()

# oversampleing code sample
# https://stackoverflow.com/questions/53273133/how-to-perform-up-sampling-using-sample-functionpy-spark

df_a = train_inital.filter(train_inital['label'] == 0)
df_b = train_inital.filter(train_inital['label'] == 1)

org_a_count = df_a.count()
org_b_count = df_b.count() 


ratio = df_a.count() / df_b.count()
# print(ratio)

df_b_overampled = df_b.sample(withReplacement=True, fraction=ratio, seed=2021)

# cahce our train values for later speed
train = df_a.unionAll(df_b_overampled).cache()

df_af = train.filter(train_inital['label'] == 0)
df_bf = train.filter(train_inital['label'] == 1)
fin_a_count = df_af.count()
fin_b_count = df_bf.count() 

print("Original No Tip Count: ", org_a_count)
print("Original Tip Count   : ", org_b_count)
print("")
print("Final No Tip Count   : ", fin_a_count)
print("Final Tip Count      : ", fin_b_count)


Original No Tip Count:  16370
Original Tip Count   :  3275

Final No Tip Count   :  16370
Final Tip Count      :  16425


In [18]:
test.show(5)
train.show(5)

+--------------------+--------------------+--------------------+------------+----------+---------------------+----------------------+----+---+------------------+----------+----------------------+------------+-----+-------------------+---------+----------+---------------+--------------+---------------+-----------------+----------+
|             Trip_ID|Trip_Start_Timestamp|  Trip_End_Timestamp|Trip_Seconds|Trip_Miles|Pickup_Community_Area|Dropoff_Community_Area|Fare|Tip|Additional_Charges|Trip_Total|Shared_Trip_Authorized|Trips_Pooled|label|      Trip_Start_TS|Trip_Year|Trip_Month|Trip_WeekNumber|Trip_DayofWeek|Trip_Start_Hour|Trip_Start_Minute|      Date|
+--------------------+--------------------+--------------------+------------+----------+---------------------+----------------------+----+---+------------------+----------+----------------------+------------+-----+-------------------+---------+----------+---------------+--------------+---------------+-----------------+----------+
|01b

Our training size has increased.  This is to be expected in upscaling.

Good reference: https://www.kaggle.com/rafjaa/resampling-strategies-for-imbalanced-datasets

In [19]:
#just as a reminder what was the truth in our test data?

dft_a = test.filter(train_inital['label'] == 0)
dft_b = test.filter(train_inital['label'] == 1)
count_test_a = dft_a.count()
count_test_b = dft_b.count()
print(count_test_a)
print(count_test_b)


4045
788


In [20]:
del (df2)

### Evaluation Functions (UDF)

In [21]:
def cmacc(pred):
    # make a confusion matrix and return the accuracy
    # select predictions and labels from prediction transform as rdd as there isn't a DF function for this
    pred_rdd= pred.select('prediction').rdd.flatMap(lambda x: x)
    label_rdd = pred.select('label').rdd.flatMap(lambda x: x)
    
    #zip them together
    predictionAndLabels =  pred_rdd.zip(label_rdd)
    
    #metrics transform
    metrics = MulticlassMetrics(predictionAndLabels)
    
    metrics2 = BinaryClassificationMetrics(predictionAndLabels)
    
    #make our confusion matrix
    cm = metrics.confusionMatrix().toArray()

    #calc accuracy from confusion matrix
    
    acc = (cm[0][0] + cm[1][1])/(cm[0][0] + cm[1][1] + cm[0][1] + cm[1][0])
    
    # McM accuracy
    
    acc2 = metrics.accuracy
    
    #calc area under curve
    auc = metrics2.areaUnderROC
    
    prc = metrics2.areaUnderPR
    
    print("Confusion Matrix")
    print(cm)
    print()
    print("Accuracy from Confusion Matrix: ", acc)
    print()
    print("Accuracy from MulticlassMetrics: ", acc2)
    print()     
    print("Area Under the ROC", auc)
    print()
    print("Area Under the PR Curve", prc)
    

In [22]:
def cmacc2(pred):
    # make a confusion matrix and return the accuracy
    # select predictions and labels from prediction transform as rdd as there isn't a DF function for this
    pred_rdd= pred.select('prediction').rdd.flatMap(lambda x: x)
    label_rdd = pred.select('label').rdd.flatMap(lambda x: x)
    
    #zip them together
    predictionAndLabels =  pred_rdd.zip(label_rdd)
    
    #metrics transform
    metrics = MulticlassMetrics(predictionAndLabels)
    
    metrics2 = BinaryClassificationMetrics(predictionAndLabels)
    
    #make our confusion matrix
    cm = metrics.confusionMatrix().toArray()

    #calc accuracy from confusion matrix
    
    acc = (cm[0][0] + cm[1][1])/(cm[0][0] + cm[1][1] + cm[0][1] + cm[1][0])
    
    #McM accuracy
    acc2 = metrics.accuracy
    
    #calc area under curve
    auc = metrics2.areaUnderROC
    
    prc = metrics2.areaUnderPR
    
    precision = metrics.precision()
    recall = metrics.recall()
    f1Score = metrics.fMeasure()
       
    print("Confusion Matrix")
    print(cm)
    print()
    print("Accuracy from Confusion Matrix: ", acc)
    print("Accuracy from MulticlassMetrics: ", acc2)
    print()
    print("Area Under the ROC", auc)
    print()
    print("Area Under the PR Curve", prc)
   
    print("Summary Stats")
    print("Precision = %s" % precision)
    print("Recall = %s" % recall)
    print("F1 Score = %s" % f1Score)
    
    # Weighted stats
    print("Weighted recall = %s" % metrics.weightedRecall)
    print("Weighted precision = %s" % metrics.weightedPrecision)
    print("Weighted F(1) Score = %s" % metrics.weightedFMeasure())
    print("Weighted F(0.5) Score = %s" % metrics.weightedFMeasure(beta=0.5))
    print("Weighted false positive rate = %s" % metrics.weightedFalsePositiveRate)

# Pipeline for LR

In [91]:
#onehotencoder to pickup
ohe_pu = OneHotEncoder(inputCol="Pickup_Community_Area", outputCol="Pickup_Community_Area_vec")

#onehotencoder to dropoff
ohe_do = OneHotEncoder(inputCol="Dropoff_Community_Area", outputCol="Dropoff_Community_Area_vec")

#onehotencoder to weekNumber
ohe_twn = OneHotEncoder(inputCol="Trip_WeekNumber", outputCol="Trip_WeekNumber_vec")

#onehotencoder to dayOfWeek
ohe_dw = OneHotEncoder(inputCol="Trip_DayofWeek", outputCol="Trip_DayofWeek_vec")

#onehotencoder to startHour
ohe_sh = OneHotEncoder(inputCol="Trip_Start_Hour", outputCol="Trip_Start_Hour_vec")

#onehotencoder to startMinute
ohe_sm = OneHotEncoder(inputCol="Trip_Start_Minute", outputCol="Trip_Start_Minute_vec")

#assemble the vector or LR

# our colulms for lr
predictor_col_for_lr = ['Trip_Seconds',
                        'Trip_Miles',
                        'Fare',
                        'Additional_Charges',
                        'Shared_Trip_Authorized',
                        'Trips_Pooled',
                        'Pickup_Community_Area_vec',
                        'Dropoff_Community_Area_vec',
                        'Trip_Year', 
                        'Trip_Month',
                        'Trip_WeekNumber_vec', 
                        'Trip_DayofWeek_vec', 
                        'Trip_Start_Hour_vec',
                        'Trip_Start_Minute_vec'
                        ] # 'Date' not supported datatype

lr_va = VectorAssembler(inputCols=predictor_col_for_lr, outputCol="features") 

#scale our LR

lr_scaler = StandardScaler(inputCol="features", outputCol="scaledFeatures",
                        withStd=True, withMean=False)

#what do we want to do if we are doing a parameter search? make the parameters as variables and just do a loop?
#we learned that this week.  May also need to add in cv step

lr = LogisticRegression(maxIter=10,
                        regParam=0.1, #org 0.1
                        elasticNetParam=0.3, #org 0.3
                        featuresCol="features",
                        labelCol="label")

# Build the pipeline
lr_pipeline = Pipeline(stages=[ohe_pu, ohe_do, ohe_twn, ohe_dw, ohe_sh, ohe_sm, lr_va, lr_scaler, lr])

# Fit the pipeline
lr_model = lr_pipeline.fit(train)

# Make a prediction
lr_prediction = lr_model.transform(test)

In [92]:
cmacc(lr_prediction)

Confusion Matrix
[[3060.  985.]
 [ 483.  305.]]

Accuracy from Confusion Matrix:  0.6962549141320091

Accuracy from MulticlassMetrics:  0.6962549141320091

Area Under the ROC 0.5717726653824675

Area Under the PR Curve 0.21394261859261904


In [93]:
cmacc2(lr_prediction)

Confusion Matrix
[[3060.  985.]
 [ 483.  305.]]

Accuracy from Confusion Matrix:  0.6962549141320091
Accuracy from MulticlassMetrics:  0.6962549141320091

Area Under the ROC 0.5717726653824675

Area Under the PR Curve 0.21394261859261904
Summary Stats
Precision = 0.6962549141320091
Recall = 0.6962549141320091
F1 Score = 0.6962549141320091
Weighted recall = 0.6962549141320091
Weighted precision = 0.7614059286433409
Weighted F(1) Score = 0.7228966007425919
Weighted F(0.5) Score = 0.7447399201039544
Weighted false positive rate = 0.5527095833670741


The above confirms the results of the first pass of the grid search from tuning below

# Pipeline for RF

In [94]:
# pipeline steps for Random Forest:

#onehotencoder to pickup
ohe_pu = OneHotEncoder(inputCol="Pickup_Community_Area", outputCol="Pickup_Community_Area_vec")

#onehotencoder to dropoff
ohe_do = OneHotEncoder(inputCol="Dropoff_Community_Area", outputCol="Dropoff_Community_Area_vec")

#onehotencoder to weekNumber
ohe_twn = OneHotEncoder(inputCol="Trip_WeekNumber", outputCol="Trip_WeekNumber_vec")

#onehotencoder to dayOfWeek
ohe_dw = OneHotEncoder(inputCol="Trip_DayofWeek", outputCol="Trip_DayofWeek_vec")

#onehotencoder to startHour
ohe_sh = OneHotEncoder(inputCol="Trip_Start_Hour", outputCol="Trip_Start_Hour_vec")

#onehotencoder to startMinute
ohe_sm = OneHotEncoder(inputCol="Trip_Start_Minute", outputCol="Trip_Start_Minute_vec")

# our colulms for rf
predictor_col_for_rf = ['Trip_Seconds',
                        'Trip_Miles',
                        'Fare',
                        'Additional_Charges',
                        'Shared_Trip_Authorized',
                        'Trips_Pooled',
                        'Pickup_Community_Area_vec',
                        'Dropoff_Community_Area_vec',
                        'Trip_Year', 
                        'Trip_Month',
                        'Trip_WeekNumber_vec', 
                        'Trip_DayofWeek_vec', 
                        'Trip_Start_Hour_vec',
                        'Trip_Start_Minute_vec'
                        ] # 'Date' not supported datatype

# assemble feature vector
rf_va = VectorAssembler(inputCols=predictor_col_for_rf, outputCol="features") 

# set classifier
rf = RandomForestClassifier(labelCol="label", 
                            featuresCol="features", 
                            numTrees=10)

# Build the pipeline
rf_pipeline = Pipeline(stages=[ohe_pu, ohe_do, ohe_twn, ohe_dw, ohe_sh, ohe_sm, rf_va, rf])

# Fit the pipeline
rf_model = rf_pipeline.fit(train)

# Make a prediction
rf_prediction = rf_model.transform(test)

# Select example rows to display.
#rf_prediction.select("prediction", "label","features").show(5) 

# Select (prediction, true label) and compute test error
rf_evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
#metric options f1|accuracy|weightedPrecision|weightedRecall

rf_accuracy = rf_evaluator.evaluate(rf_prediction)

print("Test Error = %g" % (1.0 - rf_accuracy))
print("Accuracy: " , rf_accuracy)

rfModel2 = rf_model.stages[3]
print(rfModel2)  # summary only

Test Error = 0.271467
Accuracy:  0.728533002276019
OneHotEncoder_0d071120045e


In [95]:
cmacc2(rf_prediction)

Confusion Matrix
[[3244.  801.]
 [ 511.  277.]]

Accuracy from Confusion Matrix:  0.728533002276019
Accuracy from MulticlassMetrics:  0.728533002276019

Area Under the ROC 0.5767502964743088

Area Under the PR Curve 0.2265075643254815
Summary Stats
Precision = 0.728533002276019
Recall = 0.728533002276019
F1 Score = 0.728533002276019
Weighted recall = 0.728533002276019
Weighted precision = 0.7649529611117407
Weighted F(1) Score = 0.7445812027907388
Weighted F(0.5) Score = 0.7563367617723235
Weighted false positive rate = 0.5750324093274016


# Pipeline for GBT

In [ ]:
#try a GBT 

# https://spark.apache.org/docs/latest/ml-classification-regression.html#gradient-boosted-tree-classifier

In [96]:
#onehotencoder to pickup
ohe_pu = OneHotEncoder(inputCol="Pickup_Community_Area", outputCol="Pickup_Community_Area_vec")

#onehotencoder to dropoff
ohe_do = OneHotEncoder(inputCol="Dropoff_Community_Area", outputCol="Dropoff_Community_Area_vec")

#onehotencoder to weekNumber
ohe_twn = OneHotEncoder(inputCol="Trip_WeekNumber", outputCol="Trip_WeekNumber_vec")

#onehotencoder to dayOfWeek
ohe_dw = OneHotEncoder(inputCol="Trip_DayofWeek", outputCol="Trip_DayofWeek_vec")

#onehotencoder to startHour
ohe_sh = OneHotEncoder(inputCol="Trip_Start_Hour", outputCol="Trip_Start_Hour_vec")

#onehotencoder to startMinute
ohe_sm = OneHotEncoder(inputCol="Trip_Start_Minute", outputCol="Trip_Start_Minute_vec")

# our colulms for gbt
predictor_col_for_gbt = ['Trip_Seconds',
                        'Trip_Miles',
                        'Fare',
                        'Additional_Charges',
                        'Shared_Trip_Authorized',
                        'Trips_Pooled',
                        'Pickup_Community_Area_vec',
                        'Dropoff_Community_Area_vec',
                        'Trip_Year', 
                        'Trip_Month',
                        'Trip_WeekNumber_vec', 
                        'Trip_DayofWeek_vec', 
                        'Trip_Start_Hour_vec',
                        'Trip_Start_Minute_vec'
                        ] # 'Date' not supported datatype

gbt_va = VectorAssembler(inputCols=predictor_col_for_rf, outputCol="features") 

# Automatically identify categorical features, and index them.
# # Set maxCategories so features with > 4 distinct values are treated as continuous.
# featureIndexer =\
#     VectorIndexer(inputCol="features", outputCol="indexedFeatures", maxCategories=4)#.fit(data)

# Train a GBT model.
gbt = GBTClassifier(labelCol="label", featuresCol="features", maxIter=5)

# Chain indexers and GBT in a Pipeline
gbt_pipeline = Pipeline(stages=[ohe_pu, ohe_do, ohe_twn, ohe_dw, ohe_sh, ohe_sm, gbt_va, gbt]) #labelIndexer, featureIndexer

# Train model.  This also runs the indexers.
gbt_model = gbt_pipeline.fit(train)

# Make predictions.
gbt_prediction = gbt_model.transform(test)

#gbt_prediction.show(3)

# Select example rows to display.
gbt_prediction.select("prediction", "label", "features").show(5)

# Select (prediction, true label) and compute test error
gbt_evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
gbt_accuracy = gbt_evaluator.evaluate(gbt_prediction)
print("Test Error = %g" % (1.0 - gbt_accuracy))
print('gbt accuracy = ', gbt_accuracy)
gbtModel = gbt_model.stages[3]
print(gbtModel)  # summary only

+----------+-----+--------------------+
|prediction|label|            features|
+----------+-----+--------------------+
|       0.0|  0.0|(291,[0,1,2,4,5,2...|
|       0.0|  1.0|(291,[0,1,2,3,5,1...|
|       0.0|  1.0|(291,[0,1,2,3,5,1...|
|       1.0|  0.0|(291,[0,1,2,3,5,3...|
|       0.0|  0.0|(291,[0,1,2,3,5,2...|
+----------+-----+--------------------+
only showing top 5 rows

Test Error = 0.282847
gbt accuracy =  0.7171529070970412
OneHotEncoder_c485f9d45656


In [97]:
cmacc2(gbt_prediction)

Confusion Matrix
[[3245.  800.]
 [ 567.  221.]]

Accuracy from Confusion Matrix:  0.7171529070970412
Accuracy from MulticlassMetrics:  0.7171529070970412

Area Under the ROC 0.5413409109447648

Area Under the PR Curve 0.19723951389423752
Summary Stats
Precision = 0.7171529070970412
Recall = 0.7171529070970412
F1 Score = 0.7171529070970412
Weighted recall = 0.7171529070970413
Weighted precision = 0.7477569834372434
Weighted F(1) Score = 0.7311743951823794
Weighted F(0.5) Score = 0.7408404152077188
Weighted false positive rate = 0.6344710852075116


# Pipeline LR with Tuning


In [23]:
# pipeline steps for Logistic Regression:

#onehotencoder to pickup
ohe_pu = OneHotEncoder(inputCol="Pickup_Community_Area", outputCol="Pickup_Community_Area_vec")

#onehotencoder to dropoff
ohe_do = OneHotEncoder(inputCol="Dropoff_Community_Area", outputCol="Dropoff_Community_Area_vec")

#onehotencoder to weekNumber
ohe_twn = OneHotEncoder(inputCol="Trip_WeekNumber", outputCol="Trip_WeekNumber_vec")

#onehotencoder to dayOfWeek
ohe_dw = OneHotEncoder(inputCol="Trip_DayofWeek", outputCol="Trip_DayofWeek_vec")

#onehotencoder to startHour
ohe_sh = OneHotEncoder(inputCol="Trip_Start_Hour", outputCol="Trip_Start_Hour_vec")

#onehotencoder to startMinute
ohe_sm = OneHotEncoder(inputCol="Trip_Start_Minute", outputCol="Trip_Start_Minute_vec")

#assemble the vector or LR

# our colulms for lr
predictor_col_for_lr = ['Trip_Seconds',
                        'Trip_Miles',
                        'Fare',
                        'Additional_Charges',
                        'Shared_Trip_Authorized',
                        'Trips_Pooled',
                        'Pickup_Community_Area_vec',
                        'Dropoff_Community_Area_vec',
                        'Trip_Year', 
                        'Trip_Month',
                        'Trip_WeekNumber_vec', 
                        'Trip_DayofWeek_vec', 
                        'Trip_Start_Hour_vec',
                        'Trip_Start_Minute_vec'
                        ] # 'Date' not supported datatype

lr_va = VectorAssembler(inputCols=predictor_col_for_lr, outputCol="features") 

#scale our LR

lr_scaler = StandardScaler(inputCol="features", outputCol="scaledFeatures",
                        withStd=True, withMean=False)

#what do we want to do if we are doing a parameter search? make the parameters as variables and just do a loop?
#we learned that this week.  May also need to add in cv step

lr = LogisticRegression(featuresCol="features",
                        labelCol="label") # regParam=0.1, elasticNetParam=0.3, maxIter=10,

# Build the pipeline
lr_pipeline = Pipeline(stages=[ohe_pu, ohe_do, ohe_twn, ohe_dw, ohe_sh, ohe_sm, lr_va, lr_scaler, lr])

# Set up the parameter grid
lr_paramGrid = ParamGridBuilder() \
    .addGrid(lr.regParam, [0.02, 0.03]) \
    .addGrid(lr.elasticNetParam, [0.1, 0.15]) \
    .addGrid(lr.maxIter, [5, 10]) \
    .build()

print('len(lr_paramGrid): {}'.format(len(lr_paramGrid)))
'''
25k set
best from tuning inital (accuracy):
addGrid(lr.regParam, [0.03, 0.05, 0.07]) \
.addGrid(lr.elasticNetParam, [0.15, 0.2, 0.25]) \
.addGrid(lr.maxIter, [8, 9, 10, 11, 12]) 

regParam= 0.03
elasticNetParam = 0.15
maxIter = 12
 
Confusion Matrix
[[1842. 2203.]
 [ 193.  595.]]

Accuracy from Confusion Matrix:  0.5042416718394372
Accuracy from MulticlassMetrics:  0.5042416718394372

Area Under the ROC 0.6052265753923187

Area Under the PR Curve 0.2065770273222743
Summary Stats
Precision = 0.5042416718394372
Recall = 0.5042416718394372
F1 Score = 0.5042416718394372
Weighted recall = 0.5042416718394371
Weighted precision = 0.7922492654683645
Weighted F(1) Score = 0.5612342974368173
Weighted F(0.5) Score = 0.6730989071456968
Weighted false positive rate = 0.2937885210547999

2nd round:

lr_paramGrid = ParamGridBuilder() \
    .addGrid(lr.regParam, [0.01, 0.02, 0.03]) \
    .addGrid(lr.elasticNetParam, [0.05, 0.1, 0.15]) \
    .addGrid(lr.maxIter, [11, 12, 13, 15]) \

regParam= 0.03
elasticNetParam = 0.15
maxIter = 11

Confusion Matrix
[[1815. 2230.]
 [ 183.  605.]]

Accuracy from Confusion Matrix:  0.5007241878750258
Accuracy from MulticlassMetrics:  0.5007241878750258

Area Under the ROC 0.6082342994108162

Area Under the PR Curve 0.2075564549699384
Summary Stats
Precision = 0.5007241878750258
Recall = 0.5007241878750258
F1 Score = 0.5007241878750258
Weighted recall = 0.5007241878750258
Weighted precision = 0.7950908896146499
Weighted F(1) Score = 0.5572078454446675
Weighted F(0.5) Score = 0.6716684076809212
Weighted false positive rate = 0.28425558905339354

'''

len(lr_paramGrid): 8


'\n25k set\nbest from tuning inital (accuracy):\naddGrid(lr.regParam, [0.03, 0.05, 0.07]) .addGrid(lr.elasticNetParam, [0.15, 0.2, 0.25]) .addGrid(lr.maxIter, [8, 9, 10, 11, 12]) \n\nregParam= 0.03\nelasticNetParam = 0.15\nmaxIter = 12\n \nConfusion Matrix\n[[1842. 2203.]\n [ 193.  595.]]\n\nAccuracy from Confusion Matrix:  0.5042416718394372\nAccuracy from MulticlassMetrics:  0.5042416718394372\n\nArea Under the ROC 0.6052265753923187\n\nArea Under the PR Curve 0.2065770273222743\nSummary Stats\nPrecision = 0.5042416718394372\nRecall = 0.5042416718394372\nF1 Score = 0.5042416718394372\nWeighted recall = 0.5042416718394371\nWeighted precision = 0.7922492654683645\nWeighted F(1) Score = 0.5612342974368173\nWeighted F(0.5) Score = 0.6730989071456968\nWeighted false positive rate = 0.2937885210547999\n\n2nd round:\n\nlr_paramGrid = ParamGridBuilder()     .addGrid(lr.regParam, [0.01, 0.02, 0.03])     .addGrid(lr.elasticNetParam, [0.05, 0.1, 0.15])     .addGrid(lr.maxIter, [11, 12, 13, 15])

In [24]:
# Treat the Pipeline as an Estimator, wrapping it in a CrossValidator instance.
lr_crossval = CrossValidator(estimator=lr_pipeline,
                          estimatorParamMaps=lr_paramGrid,
                          #evaluator=BinaryClassificationEvaluator(metricName='areaUnderROC'), #we can pass in our own function if necessary
                          evaluator= MulticlassClassificationEvaluator(metricName='accuracy'),
                          numFolds=5)

# you can do a custom evaluator, but it seems to be a lot of work.  https://stackoverflow.com/questions/51404344/custom-evaluator-in-pyspark
# we can use either areaUnderROC or areaUnderPR as defaults for binary.
# f1|accuracy|weightedPrecision|weightedRecall for multiclass

In [ ]:
#how to find all our items we can call
#dir(crossval.evaluator)

In [25]:
# Run cross-validation, and choose the best set of parameters. Print the training time.

t0 = time.time()
lr_cvModel = lr_crossval.setParallelism(6).fit(train) # train 6 models in parallel
print("train time:", time.time() - t0)

train time: 270.67600870132446


In [ ]:
#not sure what this metric is... apparently it is RMSE https://projector-video-pdf-converter.datacamp.com/14989/chapter4.pdf
#lr_cvModel.avgMetrics

In [ ]:
# magic code from https://stackoverflow.com/questions/36697304/how-to-extract-model-hyper-parameters-from-spark-ml-in-pyspark

# lr_cvModel.getEstimatorParamMaps()[ np.argmax(lr_cvModel.avgMetrics) ]

In [26]:
#https://dsharpc.github.io/SparkMLFlights/
#best?
#cvModel.getEstimatorParamMaps()[ np.argmin(cvModel.avgMetrics) ]

lr_cvModel.getEstimatorParamMaps()[ np.argmin(lr_cvModel.avgMetrics) ]

{Param(parent='LogisticRegression_fb4451652a14', name='regParam', doc='regularization parameter (>= 0).'): 0.03,
 Param(parent='LogisticRegression_fb4451652a14', name='elasticNetParam', doc='the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty.'): 0.15,
 Param(parent='LogisticRegression_fb4451652a14', name='maxIter', doc='max number of iterations (>= 0).'): 5}

In [27]:
lr_cvModel.getEstimatorParamMaps()[ np.argmax(lr_cvModel.avgMetrics) ]

{Param(parent='LogisticRegression_fb4451652a14', name='regParam', doc='regularization parameter (>= 0).'): 0.02,
 Param(parent='LogisticRegression_fb4451652a14', name='elasticNetParam', doc='the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty.'): 0.1,
 Param(parent='LogisticRegression_fb4451652a14', name='maxIter', doc='max number of iterations (>= 0).'): 10}

In [28]:
# Make predictions on test documents. cvModel uses the best model found (lrModel).
lr_prediction = lr_cvModel.transform(test)

In [29]:
#prediction.show(10)

In [30]:
#cmacc(lr_prediction)

In [31]:
cmacc2(lr_prediction)

Confusion Matrix
[[2056. 1989.]
 [ 244.  544.]]

Accuracy from Confusion Matrix:  0.5379681357334989
Accuracy from MulticlassMetrics:  0.5379681357334989

Area Under the ROC 0.5993185796841372

Area Under the PR Curve 0.20675778651846122
Summary Stats
Precision = 0.5379681357334989
Recall = 0.5379681357334989
F1 Score = 0.5379681357334989
Weighted recall = 0.5379681357334989
Weighted precision = 0.7831808732047225
Weighted F(1) Score = 0.5958201718109717
Weighted F(0.5) Score = 0.690207435757037
Weighted false positive rate = 0.3393309763652244


# Pipeline RF with Tuning


In [32]:
# pipeline steps for RF:

#onehotencoder to pickup
ohe_pu = OneHotEncoder(inputCol="Pickup_Community_Area", outputCol="Pickup_Community_Area_vec")

#onehotencoder to dropoff
ohe_do = OneHotEncoder(inputCol="Dropoff_Community_Area", outputCol="Dropoff_Community_Area_vec")

#onehotencoder to weekNumber
ohe_twn = OneHotEncoder(inputCol="Trip_WeekNumber", outputCol="Trip_WeekNumber_vec")

#onehotencoder to dayOfWeek
ohe_dw = OneHotEncoder(inputCol="Trip_DayofWeek", outputCol="Trip_DayofWeek_vec")

#onehotencoder to startHour
ohe_sh = OneHotEncoder(inputCol="Trip_Start_Hour", outputCol="Trip_Start_Hour_vec")

#onehotencoder to startMinute
ohe_sm = OneHotEncoder(inputCol="Trip_Start_Minute", outputCol="Trip_Start_Minute_vec")

# our colulms for rf
predictor_col_for_rf = ['Trip_Seconds',
                        'Trip_Miles',
                        'Fare',
                        'Additional_Charges',
                        'Shared_Trip_Authorized',
                        'Trips_Pooled',
                        'Pickup_Community_Area_vec',
                        'Dropoff_Community_Area_vec',
                        'Trip_Year', 
                        'Trip_Month',
                        'Trip_WeekNumber_vec', 
                        'Trip_DayofWeek_vec', 
                        'Trip_Start_Hour_vec',
                        'Trip_Start_Minute_vec'
                        ] # 'Date' not supported datatype

# assemble feature vector
rf_va = VectorAssembler(inputCols=predictor_col_for_rf, outputCol="features") 
               
# set classifier
rf = RandomForestClassifier(labelCol="label", 
                            featuresCol="features")
    
# Build the pipeline
rf_pipeline = Pipeline(stages=[ohe_pu, ohe_do, ohe_twn, ohe_dw, ohe_sh, ohe_sm, rf_va, rf])

# Set up the parameter grid
rf_paramGrid = ParamGridBuilder() \
    .addGrid(rf.numTrees, [5, 10]) \
    .addGrid(rf.maxDepth, [5, 10]) \
    .addGrid(rf.impurity, ["gini"])\
    .build()
    #.addGrid(rf.featureSubsetStrategy, ['auto', 'sqrt'])\
    
print('len(rf_paramGrid): {}'.format(len(rf_paramGrid)))

#https://medium.com/rahasak/random-forest-classifier-with-apache-spark-c63b4a23a7cc
#maxDepth, maxBins, impurity, auto and seed 
#.addGrid(randomForestClassifier.impurity, Array("entropy", "gini"))
#name='featureSubsetStrategy', auto, all, onethird, sqrt, log2, (0.0-1.0], [1-n]

'''
best from tuning inital (accuracy):
rf_paramGrid = ParamGridBuilder() \
    .addGrid(rf.numTrees, [5, 10]) \
    .addGrid(rf.maxDepth, [5, 10]) \
    .addGrid(rf.impurity, ["entropy", "gini"])\
    .build()

numTrees = 10
maxDepth = 10
impurity = gini
 
Confusion Matrix
[[2901. 1144.]
 [ 456.  332.]]

Accuracy from Confusion Matrix:  0.6689426857024623
Accuracy from MulticlassMetrics:  0.6689426857024623

Area Under the ROC 0.5692507513819781

Area Under the PR Curve 0.2070259967551608
Summary Stats
Precision = 0.6689426857024623
Recall = 0.6689426857024623
F1 Score = 0.6689426857024623
Weighted recall = 0.6689426857024622
Weighted precision = 0.7599403563099746
Weighted F(1) Score = 0.7038591473392332
Weighted F(0.5) Score = 0.735232181263078
Weighted false positive rate = 0.530441182938506

2nd attempt:

rf_paramGrid = ParamGridBuilder() \
    .addGrid(rf.numTrees, [5, 10, 15]) \
    .addGrid(rf.maxDepth, [5, 10, 15]) \
    .addGrid(rf.impurity, ["entropy", "gini"])\
    .addGrid(rf.featureSubsetStrategy, ['auto', 'sqrt'])\
    .build()


numTrees = 10
maxDepth = 15
impurity = gini
featureSubsetStrategy = auto

Confusion Matrix
[[2663. 1382.]
 [ 388.  400.]]

Accuracy from Confusion Matrix:  0.6337678460583489
Accuracy from MulticlassMetrics:  0.6337678460583489

Area Under the ROC 0.5829789236570811

Area Under the PR Curve 0.209345437091233
Summary Stats
Precision = 0.6337678460583489
Recall = 0.6337678460583489
F1 Score = 0.6337678460583489
Weighted recall = 0.6337678460583488
Weighted precision = 0.7671159775546591
Weighted F(1) Score = 0.6789410276493224
Weighted F(0.5) Score = 0.7270236282319229
Weighted false positive rate = 0.46780999874418644

'''



len(rf_paramGrid): 4


'\nbest from tuning inital (accuracy):\nrf_paramGrid = ParamGridBuilder()     .addGrid(rf.numTrees, [5, 10])     .addGrid(rf.maxDepth, [5, 10])     .addGrid(rf.impurity, ["entropy", "gini"])    .build()\n\nnumTrees = 10\nmaxDepth = 10\nimpurity = gini\n \nConfusion Matrix\n[[2901. 1144.]\n [ 456.  332.]]\n\nAccuracy from Confusion Matrix:  0.6689426857024623\nAccuracy from MulticlassMetrics:  0.6689426857024623\n\nArea Under the ROC 0.5692507513819781\n\nArea Under the PR Curve 0.2070259967551608\nSummary Stats\nPrecision = 0.6689426857024623\nRecall = 0.6689426857024623\nF1 Score = 0.6689426857024623\nWeighted recall = 0.6689426857024622\nWeighted precision = 0.7599403563099746\nWeighted F(1) Score = 0.7038591473392332\nWeighted F(0.5) Score = 0.735232181263078\nWeighted false positive rate = 0.530441182938506\n\n2nd attempt:\n\nrf_paramGrid = ParamGridBuilder()     .addGrid(rf.numTrees, [5, 10, 15])     .addGrid(rf.maxDepth, [5, 10, 15])     .addGrid(rf.impurity, ["entropy", "gini"])

In [33]:
# Treat the Pipeline as an Estimator, wrapping it in a CrossValidator instance.
rf_crossval = CrossValidator(estimator=rf_pipeline,
                          estimatorParamMaps=rf_paramGrid,
                          #evaluator=BinaryClassificationEvaluator(metricName='areaUnderROC'), #we can pass in our own function if necessary
                          evaluator= MulticlassClassificationEvaluator(metricName='accuracy'),
                          numFolds=5)

# you can do a custom evaluator, but it seems to be a lot of work.  https://stackoverflow.com/questions/51404344/custom-evaluator-in-pyspark
# we can use either areaUnderROC or areaUnderPR as defaults for binary.
# f1|accuracy|weightedPrecision|weightedRecall for multiclass

In [ ]:
#how to find all our items we can call
#dir(crossval.evaluator)

In [34]:
# Run cross-validation, and choose the best set of parameters. Print the training time.

t0 = time.time()
cvModel_rf = rf_crossval.setParallelism(6).fit(train) # train 6 models in parallel
print("train time:", time.time() - t0)

train time: 262.9041373729706


In [35]:
#not sure what this metric is... rmse
cvModel_rf.avgMetrics

[0.5679094903079406, 0.5971460644767181, 0.580450048927935, 0.6125191936476635]

In [36]:
# magic code from https://stackoverflow.com/questions/36697304/how-to-extract-model-hyper-parameters-from-spark-ml-in-pyspark

cvModel_rf.getEstimatorParamMaps()[ np.argmax(cvModel_rf.avgMetrics) ]

{Param(parent='RandomForestClassifier_0072ed8d2b6b', name='numTrees', doc='Number of trees to train (>= 1).'): 10,
 Param(parent='RandomForestClassifier_0072ed8d2b6b', name='maxDepth', doc='Maximum depth of the tree. (>= 0) E.g., depth 0 means 1 leaf node; depth 1 means 1 internal node + 2 leaf nodes.'): 10,
 Param(parent='RandomForestClassifier_0072ed8d2b6b', name='impurity', doc='Criterion used for information gain calculation (case-insensitive). Supported options: entropy, gini'): 'gini'}

In [37]:
# Make predictions on test documents. cvModel uses the best model found (lrModel).
prediction_rf = cvModel_rf.transform(test)

In [38]:
#prediction.show(10)

In [39]:
cmacc2(prediction_rf)

Confusion Matrix
[[2781. 1264.]
 [ 453.  335.]]

Accuracy from Confusion Matrix:  0.6447341195944548
Accuracy from MulticlassMetrics:  0.6447341195944548

Area Under the ROC 0.5563211773637944

Area Under the PR Curve 0.19615157769388036
Summary Stats
Precision = 0.6447341195944548
Recall = 0.6447341195944548
F1 Score = 0.6447341195944548
Weighted recall = 0.6447341195944548
Weighted precision = 0.7538776114519533
Weighted F(1) Score = 0.6852949341957646
Weighted F(0.5) Score = 0.7233605918510828
Weighted false positive rate = 0.532091764866866


In [40]:
#https://dsharpc.github.io/SparkMLFlights/
#best?
#cvModel.getEstimatorParamMaps()[ np.argmin(cvModel.avgMetrics) ]

cvModel_rf.getEstimatorParamMaps()[ np.argmin(cvModel_rf.avgMetrics) ]

{Param(parent='RandomForestClassifier_0072ed8d2b6b', name='numTrees', doc='Number of trees to train (>= 1).'): 5,
 Param(parent='RandomForestClassifier_0072ed8d2b6b', name='maxDepth', doc='Maximum depth of the tree. (>= 0) E.g., depth 0 means 1 leaf node; depth 1 means 1 internal node + 2 leaf nodes.'): 5,
 Param(parent='RandomForestClassifier_0072ed8d2b6b', name='impurity', doc='Criterion used for information gain calculation (case-insensitive). Supported options: entropy, gini'): 'gini'}

In [41]:
cvModel_rf.getEstimatorParamMaps()[ np.argmax(cvModel_rf.avgMetrics) ]

{Param(parent='RandomForestClassifier_0072ed8d2b6b', name='numTrees', doc='Number of trees to train (>= 1).'): 10,
 Param(parent='RandomForestClassifier_0072ed8d2b6b', name='maxDepth', doc='Maximum depth of the tree. (>= 0) E.g., depth 0 means 1 leaf node; depth 1 means 1 internal node + 2 leaf nodes.'): 10,
 Param(parent='RandomForestClassifier_0072ed8d2b6b', name='impurity', doc='Criterion used for information gain calculation (case-insensitive). Supported options: entropy, gini'): 'gini'}

# Pipeline GBT with Tuning

In [42]:
# pipeline steps for gbt:

#onehotencoder to pickup
ohe_pu = OneHotEncoder(inputCol="Pickup_Community_Area", outputCol="Pickup_Community_Area_vec")

#onehotencoder to dropoff
ohe_do = OneHotEncoder(inputCol="Dropoff_Community_Area", outputCol="Dropoff_Community_Area_vec")

#onehotencoder to weekNumber
ohe_twn = OneHotEncoder(inputCol="Trip_WeekNumber", outputCol="Trip_WeekNumber_vec")

#onehotencoder to dayOfWeek
ohe_dw = OneHotEncoder(inputCol="Trip_DayofWeek", outputCol="Trip_DayofWeek_vec")

#onehotencoder to startHour
ohe_sh = OneHotEncoder(inputCol="Trip_Start_Hour", outputCol="Trip_Start_Hour_vec")

#onehotencoder to startMinute
ohe_sm = OneHotEncoder(inputCol="Trip_Start_Minute", outputCol="Trip_Start_Minute_vec")

# our colulms for gbt
predictor_col_for_gbt = ['Trip_Seconds',
                        'Trip_Miles',
                        'Fare',
                        'Additional_Charges',
                        'Shared_Trip_Authorized',
                        'Trips_Pooled',
                        'Pickup_Community_Area_vec',
                        'Dropoff_Community_Area_vec',
                        'Trip_Year', 
                        'Trip_Month',
                        'Trip_WeekNumber_vec', 
                        'Trip_DayofWeek_vec', 
                        'Trip_Start_Hour_vec',
                        'Trip_Start_Minute_vec'
                        ] # 'Date' not supported datatype

gbt_va = VectorAssembler(inputCols=predictor_col_for_gbt, outputCol="features") 

# Train a GBT model.
gbt = GBTClassifier(labelCol="label", featuresCol="features") #, maxIter=5

# Chain indexers and GBT in a Pipeline
gbt_pipeline = Pipeline(stages=[ohe_pu, ohe_do, ohe_twn, ohe_dw, ohe_sh, ohe_sm, gbt_va, gbt]) #labelIndexer, featureIndexer

# Set up the parameter grid
gbt_paramGrid = ParamGridBuilder() \
    .addGrid(gbt.maxIter, [5, 10])\
    .addGrid(gbt.maxDepth, [5, 10])\
    .build()

print('len(gbt_paramGrid): {}'.format(len(gbt_paramGrid)))


'''
best from tuning inital (accuracy):
gbt_paramGrid = ParamGridBuilder() \
    .addGrid(gbt.maxIter, [5, 10, 20]) \
    .build()

maxIter = 20

Confusion Matrix
[[2798. 1247.]
 [ 423.  365.]]

Accuracy from Confusion Matrix:  0.654458928201945
Accuracy from MulticlassMetrics:  0.654458928201945

Area Under the ROC 0.5774580700620556

Area Under the PR Curve 0.2094152550126291
Summary Stats
Precision = 0.654458928201945
Recall = 0.654458928201945
F1 Score = 0.654458928201945
Weighted recall = 0.654458928201945
Weighted precision = 0.7639586098089827
Weighted F(1) Score = 0.6941837869282138
Weighted F(0.5) Score = 0.7327747544723259
Weighted false positive rate = 0.4995427880778336


maxIter = 40

Confusion Matrix
[[2571. 1474.]
 [ 382.  406.]]

Accuracy from Confusion Matrix:  0.6159735154148562
Accuracy from MulticlassMetrics:  0.6159735154148562

Area Under the ROC 0.5754139659791808

Area Under the PR Curve 0.20313239804234073
Summary Stats
Precision = 0.6159735154148562
Recall = 0.6159735154148562
F1 Score = 0.6159735154148562
Weighted recall = 0.6159735154148562
Weighted precision = 0.7638968296438198
Weighted F(1) Score = 0.6646010165217533
Weighted F(0.5) Score = 0.7183436330713325
Weighted false positive rate = 0.4651455834564943


gbt_paramGrid = ParamGridBuilder() \
    .addGrid(gbt.maxIter, [5, 10])\
    .addGrid(gbt.maxDepth, [5, 10])\
    .build()
    
maxIter = 5
maxDepth = 5

Confusion Matrix
[[2863. 1182.]
 [ 469.  319.]]

Accuracy from Confusion Matrix:  0.6583902338092282
Accuracy from MulticlassMetrics:  0.6583902338092282

Area Under the ROC 0.5563048634335804

Area Under the PR Curve 0.19780050930331394
Summary Stats
Precision = 0.6583902338092282
Recall = 0.6583902338092282
F1 Score = 0.6583902338092282
Weighted recall = 0.6583902338092282
Weighted precision = 0.7537989743798752
Weighted F(1) Score = 0.6950856095348379
Weighted F(0.5) Score = 0.7279222226014918
Weighted false positive rate = 0.5457805069420676
'''



len(gbt_paramGrid): 4


'\nbest from tuning inital (accuracy):\ngbt_paramGrid = ParamGridBuilder()     .addGrid(gbt.maxIter, [5, 10, 20])     .build()\n\nmaxIter = 20\n\nConfusion Matrix\n[[2798. 1247.]\n [ 423.  365.]]\n\nAccuracy from Confusion Matrix:  0.654458928201945\nAccuracy from MulticlassMetrics:  0.654458928201945\n\nArea Under the ROC 0.5774580700620556\n\nArea Under the PR Curve 0.2094152550126291\nSummary Stats\nPrecision = 0.654458928201945\nRecall = 0.654458928201945\nF1 Score = 0.654458928201945\nWeighted recall = 0.654458928201945\nWeighted precision = 0.7639586098089827\nWeighted F(1) Score = 0.6941837869282138\nWeighted F(0.5) Score = 0.7327747544723259\nWeighted false positive rate = 0.4995427880778336\n\n\nmaxIter = 40\n\nConfusion Matrix\n[[2571. 1474.]\n [ 382.  406.]]\n\nAccuracy from Confusion Matrix:  0.6159735154148562\nAccuracy from MulticlassMetrics:  0.6159735154148562\n\nArea Under the ROC 0.5754139659791808\n\nArea Under the PR Curve 0.20313239804234073\nSummary Stats\nPrecisi

In [43]:

# Treat the Pipeline as an Estimator, wrapping it in a CrossValidator instance.
gbt_crossval = CrossValidator(estimator=gbt_pipeline,
                          estimatorParamMaps=gbt_paramGrid,
                          #evaluator=BinaryClassificationEvaluator(metricName='areaUnderROC'), #we can pass in our own function if necessary
                          evaluator= MulticlassClassificationEvaluator(metricName='accuracy'),
                          numFolds=5)

# you can do a custom evaluator, but it seems to be a lot of work.  https://stackoverflow.com/questions/51404344/custom-evaluator-in-pyspark
# we can use either areaUnderROC or areaUnderPR as defaults for binary.
# f1|accuracy|weightedPrecision|weightedRecall for multiclass

In [ ]:
#how to find all our items we can call
#dir(crossval.evaluator)

In [44]:
# Run cross-validation, and choose the best set of parameters. Print the training time.

t0 = time.time()
cvModel_gbt = gbt_crossval.setParallelism(6).fit(train) # train 6 models in parallel
print("train time:", time.time() - t0)

train time: 836.6635949611664


In [45]:
#not sure what this metric is...
cvModel_gbt.avgMetrics

[0.596076567228641, 0.64811843790031, 0.6117758066421406, 0.6769369131046441]

In [46]:
# magic code from https://stackoverflow.com/questions/36697304/how-to-extract-model-hyper-parameters-from-spark-ml-in-pyspark

#cvModel_gbt.getEstimatorParamMaps()[ np.argmax(cvModel_rf.avgMetrics) ]

cvModel_gbt.getEstimatorParamMaps()[np.argmax(cvModel_gbt.avgMetrics)]

{Param(parent='GBTClassifier_6ec4c793ba20', name='maxIter', doc='max number of iterations (>= 0).'): 10,
 Param(parent='GBTClassifier_6ec4c793ba20', name='maxDepth', doc='Maximum depth of the tree. (>= 0) E.g., depth 0 means 1 leaf node; depth 1 means 1 internal node + 2 leaf nodes.'): 10}

In [47]:
cvModel_gbt.getEstimatorParamMaps()[np.argmin(cvModel_gbt.avgMetrics)]

{Param(parent='GBTClassifier_6ec4c793ba20', name='maxIter', doc='max number of iterations (>= 0).'): 5,
 Param(parent='GBTClassifier_6ec4c793ba20', name='maxDepth', doc='Maximum depth of the tree. (>= 0) E.g., depth 0 means 1 leaf node; depth 1 means 1 internal node + 2 leaf nodes.'): 5}

In [48]:
# Make predictions on test documents. cvModel uses the best model found (lrModel).
prediction_gbt = cvModel_gbt.transform(test)

In [49]:
#prediction.show(10)

In [50]:
cmacc2(prediction_gbt)

Confusion Matrix
[[2618. 1427.]
 [ 389.  399.]]

Accuracy from Confusion Matrix:  0.6242499482722946
Accuracy from MulticlassMetrics:  0.6242499482722946

Area Under the ROC 0.5767819831464551

Area Under the PR Curve 0.20482020238384122
Summary Stats
Precision = 0.6242499482722946
Recall = 0.6242499482722946
F1 Score = 0.6242499482722946
Weighted recall = 0.6242499482722946
Weighted precision = 0.7643090256415888
Weighted F(1) Score = 0.6711999721980451
Weighted F(0.5) Score = 0.7218205677729161
Weighted false positive rate = 0.47068598197938427
